### poscarのパスのリストをload

In [1]:
import numpy as np

In [6]:
poscar_path_list = np.load('poscar_path_list.npy', allow_pickle=True)

In [4]:
# from tqdm import tqdm
# import numpy as np
# poscar_path_list = list(tqdm(np.load('poscar_path_list.npy', allow_pickle=True)))


  0%|                                                                                                                                         | 0/308325 [00:44<?, ?it/s]


In [6]:
print(f"len(poscar_path_list) -> {len(poscar_path_list)}")

len(poscar_path_list) -> 308325


### poscar2nnlistをsubprocessで実行する

#### poscar2nnlistをビルド

In [11]:
!ls 

__pycache__			  mk_poscar_by_c2c.ipynb
error.txt			  mk_poscar_by_c2c.py
get_cif_pathes.ipynb		  mk_poscar_by_c2c_test.ipynb
get_cif_pathes.py		  my_package
get_poacar_path_list.ipynb	  neib_code.zip
get_poacar_path_list.py		  others
mk_nnlist_by_poscar2nnlist.ipynb  poscar_path_list.npy


In [12]:
!unzip neib_code.zip

Archive:  neib_code.zip
   creating: neib_code/
chmod (directory attributes) error: Operation not permitted
  inflating: neib_code/neib.f90      fchmod (file attributes) error: Operation not permitted
 (warning) cannot set modif./access times
          Operation not permitted
  inflating: neib_code/m_ftox.f90    fchmod (file attributes) error: Operation not permitted
 (warning) cannot set modif./access times
          Operation not permitted
  inflating: neib_code/sample_code   fchmod (file attributes) error: Operation not permitted
 (warning) cannot set modif./access times
          Operation not permitted
  inflating: neib_code/ABW.cif       fchmod (file attributes) error: Operation not permitted
 (warning) cannot set modif./access times
          Operation not permitted
          Operation not permitted
          Operation not permitted
     failed setting times/attribs for 1 dir entries

In [13]:
!ls

__pycache__			  mk_poscar_by_c2c.py
error.txt			  mk_poscar_by_c2c_test.ipynb
get_cif_pathes.ipynb		  my_package
get_cif_pathes.py		  neib_code
get_poacar_path_list.ipynb	  neib_code.zip
get_poacar_path_list.py		  others
mk_nnlist_by_poscar2nnlist.ipynb  poscar_path_list.npy
mk_poscar_by_c2c.ipynb


In [14]:
!ls neib_code/

ABW.cif  m_ftox.f90  neib.f90  sample_code


In [ ]:
# !gfortran m_ftox.f90 neib.f90 -o poscar2nnlist

### poscar2nnlistをsubprocessで使用

linuxのターミナルでのposcar2nnlistの実行コマンド
$ ./poscar2nnlist POSCAR 10

In [17]:
import subprocess

In [22]:
! ./neib_code/poscar2nnlist ./neib_code/POSCAR 3

At line 145 of file neib.f90
Fortran runtime error: Cannot open file '.': Is a directory

Error termination. Backtrace:
#0  0x7f5f0e7d6960 in ???
#1  0x7f5f0e7d74d9 in ???
#2  0x7f5f0ea26ee6 in ???
#3  0x5620dce58636 in MAIN__
#4  0x5620dce5ac71 in main


In [19]:
poscar_path = './neib_code/POSCAR'
subprocess.run(['./neib_code/poscar2nnlist', poscar_path, '3'])

At line 145 of file neib.f90
Fortran runtime error: Cannot open file '.': Is a directory

Error termination. Backtrace:
#0  0x7fce87700960 in ???
#1  0x7fce877014d9 in ???
#2  0x7fce87950ee6 in ???
#3  0x56186a896636 in MAIN__
#4  0x56186a898c71 in main


CompletedProcess(args=['./neib_code/poscar2nnlist', './neib_code/POSCAR', '3'], returncode=2)

#### 上より，poscar2nnlistでPOSCARファイルを指定するとき，ディレクトリを挟むと実行できない？もよう

#### よってPOSCARのあるディレクトリにposcar2nnlistファイルをコピーする

In [25]:
import os

In [28]:
poscar_folder_path = [os.path.split(p)[0] for p in poscar_path_list]

In [30]:
poscar_folder_path[0:5]

['../cif/1/00/00/1000003',
 '../cif/1/00/00/1000005',
 '../cif/1/00/00/1000006',
 '../cif/1/00/00/1000002',
 '../cif/1/00/00/1000001']

In [56]:
!ls ../cif/1/00/00/1000003

POSCAR	do_poscar2nnlist.ipynb	do_poscar2nnlist.sh  poscar2nnlist


In [51]:
!../cif/1/00/00/1000003/poscar2nnlist ../cif/1/00/00/1000003/POSCAR 2

At line 145 of file neib.f90
Fortran runtime error: Cannot open file '..': Is a directory

Error termination. Backtrace:
#0  0x7f9fe5dd2960 in ???
#1  0x7f9fe5dd34d9 in ???
#2  0x7f9fe6022ee6 in ???
#3  0x55bdd7f72636 in MAIN__
#4  0x55bdd7f74c71 in main


In [45]:
!ls ../cif/1/00/00/1000003

POSCAR	poscar2nnlist


In [ ]:
# !sudo cp ./neib_code/poscar2nnlist ../cif/1/00/00/1000003

In [ ]:
./poscar2nnlist POSCAR -2

In [32]:
import shutil

In [38]:
src = './neib_code/poscar2nnlist'
dst = poscar_folder_path[0]
dst = '.'
shutil.copy2(src, dst)

PermissionError: [Errno 1] Operation not permitted

In [ ]:
try:
    subprocess.run(['cif2cell', cif_path, '-p', 'vasp', '-o', poscar_path, '--vasp-format=5'])
except Exception as e:
    error_list.append(e)

In [66]:
!ls ../cif/1/00/00/1000003

POSCAR			do_poscar2nnlist.sh  m_ftox.mod  neib.mod
do_poscar2nnlist.ipynb	m_ftox.f90	     neib.f90	 poscar2nnlist


In [67]:
!bash ../cif/1/00/00/1000003/do_poscar2nnlist.sh

At line 146 of file neib.f90
Fortran runtime error: End of file

Error termination. Backtrace:
#0  0x7f239d24f960 in ???
#1  0x7f239d2504d9 in ???
#2  0x7f239d4a417b in ???
#3  0x7f239d4a7fd7 in ???
#4  0x7f239d4a80e3 in ???
#5  0x5573fc61e69d in MAIN__
#6  0x5573fc620c71 in main


In [58]:
import subprocess
subprocess.run(['bash', '../cif/1/00/00/1000003/do_poscar2nnlist.sh'])

At line 146 of file neib.f90
Fortran runtime error: End of file

Error termination. Backtrace:
#0  0x7f5947bac960 in ???
#1  0x7f5947bad4d9 in ???
#2  0x7f5947e0117b in ???
#3  0x7f5947e04fd7 in ???
#4  0x7f5947e050e3 in ???
#5  0x55616b3c069d in MAIN__
#6  0x55616b3c2c71 in main


CompletedProcess(args=['bash', '../cif/1/00/00/1000003/do_poscar2nnlist.sh'], returncode=2)

In [68]:
!ls ../cif/1/00/00/1000003

POSCAR			do_poscar2nnlist.sh  neib.f90
do_poscar2nnlist.ipynb	m_ftox.f90	     neib.mod
do_poscar2nnlist.py	m_ftox.mod	     poscar2nnlist


In [70]:
import subprocess

In [71]:
subprocess.run(['python3', '../cif/1/00/00/1000003/do_poscar2nnlist.py'])

At line 146 of file neib.f90
Fortran runtime error: End of file

Error termination. Backtrace:
#0  0x7f66e8c20960 in ???
#1  0x7f66e8c214d9 in ???
#2  0x7f66e8e7517b in ???
#3  0x7f66e8e78fd7 in ???
#4  0x7f66e8e790e3 in ???
#5  0x5654cb63c69d in MAIN__
#6  0x5654cb63ec71 in main


CompletedProcess(args=['python3', '../cif/1/00/00/1000003/do_poscar2nnlist.py'], returncode=0)

In [73]:
!python3 ../cif/1/00/00/1000003/do_poscar2nnlist.py

At line 146 of file neib.f90
Fortran runtime error: End of file

Error termination. Backtrace:
#0  0x7f16ddc5b960 in ???
#1  0x7f16ddc5c4d9 in ???
#2  0x7f16ddeb017b in ???
#3  0x7f16ddeb3fd7 in ???
#4  0x7f16ddeb40e3 in ???
#5  0x55d2a577c69d in MAIN__
#6  0x55d2a577ec71 in main


In [72]:
!ls ../cif/1/00/00/1000003

POSCAR			do_poscar2nnlist.sh  neib.f90
do_poscar2nnlist.ipynb	m_ftox.f90	     neib.mod
do_poscar2nnlist.py	m_ftox.mod	     poscar2nnlist


### poscarファイルの絶対パスを作成

In [145]:
poscar_folder_abs_path = ['/mnt/ssd_elecom_black_c2c' + p.split('..')[1] for p in poscar_folder_path]

#### 1. Change current dir to dir that exists a POSCAR file
#### 2. Run poscar2nnlist
#### 3. Repeat (1.->2.) workflow

In [110]:
!pwd

/mnt/ssd_elecom_black_c2c/cif/1/00/00/1000003


In [109]:
subprocess.run(['ls', poscar_folder_path[1]]) 

ls: cannot access '../cif/1/00/00/1000005': No such file or directory


CompletedProcess(args=['ls', '../cif/1/00/00/1000005'], returncode=2)

In [146]:
import os
import subprocess
def cd_dir_and_pos2nnlist(poscar_folder_path):
    # 1. Change current dir to dir that exists a POSCAR file
    os.chdir(poscar_folder_path)
    
    # 2. Run poscar2nnlist
    try:
        subprocess.run(['/mnt/ssd_elecom_black_c2c/ssd_elecom_black_c2c-script/neib_code/poscar2nnlist', 'POSCAR', '2'])
    except Exception as e:
        pass

In [ ]:
joblib

### 並列化

In [151]:
from multiprocessing import Pool, cpu_count
# from tqdm import tqdm
from tqdm import tqdm
import time

before = time.time()
try:
    p = Pool(cpu_count() - 1)
    tqdm(p.imap(cd_dir_and_pos2nnlist, poscar_folder_abs_path[0:10]), total=len(poscar_folder_abs_path[0:10]))
    
finally:
    p.close()
    p.join()
after = time.time()
print(f"it took {after - before}sec.")

  0%|                                                                                                                                             | 0/10 [00:00<?, ?it/s]


4 88 100 8 32  !nsite
 readin type           1 Cl           4
 readin type           2 C          88
 readin type           3 H         100
 readin type           4 N           8
 readin type           5 O          32
direct  !postype
4 12 12 28  !nsite
 readin type           1 Sr           4
 readin type           2 D          12
 readin type           3 C          12
 readin type           4 O          28
direct  !postype
4 58 2 2 60  !nsite
 readin type           1 P           4
 readin type           2 C          58
 readin type           3 Cu           2
 readin type           4 I           2
 readin type           5 H          60
direct  !postype
6.3410000000000002  ! unit (angstrom)
1 Sr 0.878600  0.418400  0.738500
2 Sr 0.378600  0.081600  0.238500
12 12 20 24 64  !nsite
3 Sr 0.621400  0.918400  0.761500
4 Sr 0.121400  0.581600  0.261500
 readin type           1 H          12
 readin type           2 V          12
 readin type           3 Sr          20
 readin type           4

In [76]:
print(os.getcwd())

'/mnt/ssd_elecom_black_c2c/ssd_elecom_black_c2c-script'

In [77]:
os.chdir(poscar_folder_path[0])

In [78]:
print(os.getcwd())

/mnt/ssd_elecom_black_c2c/cif/1/00/00/1000003


In [79]:
!ls

POSCAR			do_poscar2nnlist.sh  neib.f90
do_poscar2nnlist.ipynb	m_ftox.f90	     neib.mod
do_poscar2nnlist.py	m_ftox.mod	     poscar2nnlist


In [80]:
!./poscar2nnlist POSCAR 2

4 8  !nsite
 readin type           1 Si           4
 readin type           2 O           8
direct  !postype
9.8729999999999993  ! unit (angstrom)
1 Si 0.942100  0.849300  0.801400
2 Si 0.743500  0.150700  0.198600
3 Si 0.057900  0.150700  0.198600
4 Si 0.256500  0.849300  0.801400
5 O 0.147700  0.897700  0.704600
6 O 0.852300  0.102300  0.295400
7 O 0.810000  0.500000  0.000000
8 O 0.810000  0.000000  0.000000
9 O 0.190000  0.000000  0.000000
10 O 0.190000  0.500000  0.000000
11 O 0.000000  0.000000  0.500000
12 O 0.500000  0.000000  0.500000
 2.000000  !rmax (angstrom)
1.000000  0.000000  0.000000  !plat(:,1)
0.000000  0.532158  0.000000  !plat(:,2)
0.500000  0.266079  0.444141  !plat(:,3)
12  !natom
1.342800  0.665198  0.355934 pos atomid= 1 Si   !Atom position: cartetian
0.842800  0.133040  0.088206 pos atomid= 2 Si   !Atom position: cartetian
0.157200  0.133040  0.088206 pos atomid= 3 Si   !Atom position: cartetian
0.657200  0.665198  0.355934 pos atomid= 4 Si   !Atom position: car

In [84]:
!ls

POSCAR			do_poscar2nnlist.py  m_ftox.mod  poscar2nnlist
POSCAR.nnlist		do_poscar2nnlist.sh  neib.f90
do_poscar2nnlist.ipynb	m_ftox.f90	     neib.mod


In [85]:
!rm POSCAR.nnlist

In [86]:
!ls

POSCAR			do_poscar2nnlist.sh  neib.f90
do_poscar2nnlist.ipynb	m_ftox.f90	     neib.mod
do_poscar2nnlist.py	m_ftox.mod	     poscar2nnlist


<!-- #### （1. poscarのあるdirにposcar2nnllistをcopy -> 2．change current dir -> 3. poscar2nnlistを実行） を繰り返す -->

In [88]:
!/mnt/ssd_elecom_black_c2c/ssd_elecom_black_c2c-script/neib_code/poscar2nnlist POSCAR 2

4 8  !nsite
 readin type           1 Si           4
 readin type           2 O           8
direct  !postype
9.8729999999999993  ! unit (angstrom)
1 Si 0.942100  0.849300  0.801400
2 Si 0.743500  0.150700  0.198600
3 Si 0.057900  0.150700  0.198600
4 Si 0.256500  0.849300  0.801400
5 O 0.147700  0.897700  0.704600
6 O 0.852300  0.102300  0.295400
7 O 0.810000  0.500000  0.000000
8 O 0.810000  0.000000  0.000000
9 O 0.190000  0.000000  0.000000
10 O 0.190000  0.500000  0.000000
11 O 0.000000  0.000000  0.500000
12 O 0.500000  0.000000  0.500000
 2.000000  !rmax (angstrom)
1.000000  0.000000  0.000000  !plat(:,1)
0.000000  0.532158  0.000000  !plat(:,2)
0.500000  0.266079  0.444141  !plat(:,3)
12  !natom
1.342800  0.665198  0.355934 pos atomid= 1 Si   !Atom position: cartetian
0.842800  0.133040  0.088206 pos atomid= 2 Si   !Atom position: cartetian
0.157200  0.133040  0.088206 pos atomid= 3 Si   !Atom position: cartetian
0.657200  0.665198  0.355934 pos atomid= 4 Si   !Atom position: car

In [13]:
import numpy as np
import os
import subprocess
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import time


poscar_path_list = np.load('poscar_path_list.npy', allow_pickle=True)
poscar_folder_path = [os.path.split(p)[0] for p in poscar_path_list]
poscar_folder_abs_path = ['/mnt/ssd_elecom_black_c2c' + p.split('..')[1] for p in poscar_folder_path]


def cd_dir_and_pos2nnlist(poscar_folder_path):
    # 1. Change current dir to dir that exists a POSCAR file
    os.chdir(poscar_folder_path)
    
    # 2. Run poscar2nnlist
    try:
        subprocess.run(['/mnt/ssd_elecom_black_c2c/ssd_elecom_black_c2c-script/neib_code/poscar2nnlist', 'POSCAR', '2'], stdout=subprocess.DEVNULL)
    except Exception as e:
        pass


before = time.time()
try:
    p = Pool(cpu_count() - 1)
    list(tqdm(p.imap(cd_dir_and_pos2nnlist, poscar_folder_abs_path[0:10]), total=len(poscar_folder_abs_path[0:10])))
    
finally:
    p.close()
    p.join()
after = time.time()
print(f"it took {after - before}sec.")


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 13.06it/s]

it took 0.873424768447876sec.


In [3]:
import numpy as np
import os
import subprocess
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import time


poscar_path_list = np.load('poscar_path_list.npy', allow_pickle=True)
print(f"poscar_path_list is loaded from .npy file.")
poscar_folder_path = [os.path.split(p)[0] for p in poscar_path_list]
poscar_folder_abs_path = ['/mnt/ssd_elecom_black_c2c' + p.split('..')[1] for p in poscar_folder_path]
print(f"poscar_folder_abs_path is completely made.")

def cd_dir_and_pos2nnlist(poscar_folder_path):
    # 1. Change current dir to dir that exists a POSCAR file
    os.chdir(poscar_folder_path)
    
    # 2. Run poscar2nnlist
    try:
        subprocess.run(['/mnt/ssd_elecom_black_c2c/ssd_elecom_black_c2c-script/neib_code/poscar2nnlist', 'POSCAR', '2'], stdout=subprocess.DEVNULL)
    except Exception as e:
        pass


before = time.time()
try:
    p = Pool(cpu_count() - 1)
    for path in tqdm(poscar_folder_abs_path[0:10], total=len(poscar_folder_abs_path[0:10])):
        cd_dir_and_pos2nnlist(path)
    
finally:
    p.close()
    p.join()
after = time.time()
print(f"it took {after - before}sec.")


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.32it/s]


it took 1.9915390014648438sec.


In [9]:
os.chdir('/mnt/ssd_elecom_black_c2c/ssd_elecom_black_c2c-script/')

In [10]:
os.getcwd()

'/mnt/ssd_elecom_black_c2c/ssd_elecom_black_c2c-script'

In [12]:
import numpy as np
import os
import subprocess
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import time


poscar_path_list = np.load('poscar_path_list.npy', allow_pickle=True)
poscar_folder_path = [os.path.split(p)[0] for p in poscar_path_list]
poscar_folder_abs_path = ['/mnt/ssd_elecom_black_c2c' + p.split('..')[1] for p in poscar_folder_path]


def cd_dir_and_pos2nnlist(poscar_folder_path):
    # 1. Change current dir to dir that exists a POSCAR file
    os.chdir(poscar_folder_path)
    
    # 2. Run poscar2nnlist
    try:
        subprocess.run(['/mnt/ssd_elecom_black_c2c/ssd_elecom_black_c2c-script/neib_code/poscar2nnlist', 'POSCAR', '2'], stdout=subprocess.DEVNULL)
    except Exception as e:
        pass


before = time.time()
try:
    p = Pool(cpu_count() - 1)
    with tqdm(total=len(poscar_folder_abs_path[0:10])) as t:
        p.imap(cd_dir_and_pos2nnlist, poscar_folder_abs_path[0:10])
    
finally:
    p.close()
    p.join()
after = time.time()
print(f"it took {after - before}sec.")


  0%|                                                                                                                                 | 0/10 [00:00<?, ?it/s]


it took 1.499279499053955sec.
